In [ ]:
baizer::pkglib(baizer, tidyverse, ggsci, ggbeeswarm, plutor, ComplexHeatmap, plotly, ggplotify, echarts4r, jsonlite, knitr, yaml)

In [ ]:
sample <- snakemake@wildcards[['sample']]

# replace default configs with sample configs
config <- replace_item(snakemake@config, snakemake@config[[sample]])

In [ ]:
filter_dir <- snakemake@input[['filter_dir']]
stat_dir <- snakemake@params[['stat_dir']]

In [ ]:
# theme and colors
mytheme <- readLines(snakemake@params[['echarts_theme']]) %>% str_c(collapse='')
theme_list <- fromJSON(snakemake@params[['echarts_theme']])
Vcol <- theme_list$color
grey_col <- '#f5f5f5'

In [ ]:
# list to save all results
Lres <- list()

In [ ]:
for (flt_mode in names(config[['flt_mode']])) {
    if (str_c(flt_mode, '_flt.csv') %in% dir(stat_dir)) {
        TBkeep <- read_csv(str_glue('{filter_dir}/{flt_mode}_keep.csv'))
        TBflt <- read_csv(str_glue('{filter_dir}/{flt_mode}_flt.csv'))
        
        Lres[[flt_mode]] <- list(df=TBflt, keep=TBkeep)
    }
}

In [ ]:
# filter

In [ ]:
for (flt_mode in names(Lres)) {
    TBflt <- Lres[[flt_mode]][['df']]
    TBkeep <- Lres[[flt_mode]][['keep']]
    
    TBkeep <- TBkeep %>% c2r('cell') %>% seriate_df %>% r2c('cell')
    TBkeep_longer <- TBkeep %>% pivot_longer(-cell, names_to='term', values_to='value')

    e1 <- TBkeep_longer %>% 
      mutate(term=str_replace(term, '_keep$', '')) %>%
      e_charts(term) %>% 
      e_heatmap(cell, value) %>%
      e_x_axis(axisLabel=list(interval=0, rotate=45)) %>%
      e_y_axis(name='Cell', nameLocation='center', 
               axisLabel=list(show=FALSE), axisTick=list(show=FALSE),
              ) %>%
      e_visual_map(value, type='piecewise', splitNumber=2, orient='horizontal', top='top', left='center',
                    pieces=list(list(min=0, max=0, label='removed'), 
                                list(min=1, max=1, label='kept'))) %>%
      e_theme_custom(mytheme)

    e2 <- as_tibble(TBkeep %>% select(-cell) %>% apply(2, sum), rownames='term') %>%
      mutate(term=str_replace(term, '_keep$', '')) %>%
      e_charts(term) %>%
      e_bar(value) %>%
      e_x_axis(axisLabel=list(interval=0, rotate=45)) %>%
      e_y_axis(name='Kept cells') %>%
      e_mark_line(data = list(yAxis = nrow(TBflt)), title = "filtered") %>%
      e_legend(show=FALSE) %>%
      e_theme_custom(mytheme)

    Lres[[flt_mode]][['filter']] <- e_arrange(e1, e2, cols=2)
    
}

In [ ]:
# hashtag

In [ ]:
for (flt_mode in names(Lres)) {
    if (('FB' %in% config[['flt_mode']][[flt_mode]]) && (length(config[[sample]][['id2seq']][['HT']]) > 0)) {
        
        TBflt <- Lres[[flt_mode]][['df']]
        TBfb_ht <- TBflt %>% select(cell, matches('^HT_'), sum_HT)

        e1 <- TBfb_ht %>% 
            group_by(HT_group) %>% 
            e_charts(sum_HT) %>% 
            e_scatter(HT_ratio, symbol_size=10) %>%
            e_x_axis(name='All hashtag umi', nameLocation='middle', nameGap=30) %>%
            e_y_axis(name='Major hashtag ratio', nameLocation='middle', nameGap=30) %>%
            e_tooltip(trigger = 'axis') %>%
            e_theme_custom(mytheme)

        e2 <- TBfb_ht %>% count(HT_group) %>%
          e_charts(HT_group) %>%
          e_pie(n, radius = c("50%", "70%")) %>%
          e_theme_custom(mytheme)

        Lres[[flt_mode]][['HT']] <- e_arrange(e1, e2, cols=2)
    }
}

In [ ]:
# binding feature barcode

In [ ]:
for (flt_mode in names(Lres)) {
    if (('FB' %in% config[['flt_mode']][[flt_mode]]) && (length(config[[sample]][['id2seq']][['BD']]) > 0)) {

        # BD ratio
        TBflt <- Lres[[flt_mode]][['df']]
        TBp <- TBflt %>% select(cell, matches('^BD_[^#]+_ratio$')) %>% 
            c2r('cell') %>% seriate_df %>% r2c('cell') %>% 
            pivot_longer(-cell, names_to='x', values_to='v') %>%
            mutate(x=str_replace(x, 'BD_(.+?)_ratio', '\\1'))

        e1 <- TBp %>% 
          e_charts(x) %>% 
          e_heatmap(cell, v) %>%
          e_x_axis(axisLabel=list(interval=0)) %>%
          e_y_axis(name='Cell', nameLocation='center', 
                   axisLabel=list(show=FALSE), axisTick=list(show=FALSE),
                  ) %>%
          e_visual_map(v, text=c('ratio', ''), orient='horizontal', top='top', 
                       left='center', precision=2) %>%
          e_theme_custom(mytheme)

        # BD type
        e2 <- TBflt %>% count(BD_type) %>%
          e_charts(BD_type) %>%
          e_pie(n, radius = c("50%", "70%"), label=list(show=TRUE, formatter='{b}({c}, {d}%)'))

        # absolute value
        TBp <- TBflt %>% select(cell, matches('^BD_[^#]+$'), -matches('_ratio$'), -BD_type) %>%
            c2r('cell') %>% seriate_df %>% r2c('cell') %>% 
            pivot_longer(-cell, names_to='x', values_to='v') %>%
            mutate(x=str_replace(x, 'BD_', ''))

        e3 <- TBp %>% 
          e_charts(x) %>% 
          e_heatmap(cell, v) %>%
          e_x_axis(axisLabel=list(interval=0)) %>%
          e_y_axis(name='Cell', nameLocation='center', 
                   axisLabel=list(show=FALSE), axisTick=list(show=FALSE),
                  ) %>%
          e_visual_map(v, text=c('umi', ''), orient='horizontal', top='top', left='center') %>%
          e_theme_custom(mytheme)

        TBp <- TBflt %>% select(cell, all_of(str_c('BD_', names(config[[sample]][['id2seq']][['BD']])))) %>%
            c2r('cell') %>% seriate_df %>% r2c('cell') %>% 
            pivot_longer(-cell, names_to='x', values_to='v') %>%
            mutate(x=str_replace(x, 'BD_', ''))
        e4 <- TBp %>% 
          e_charts(x) %>% 
          e_heatmap(cell, v) %>%
          e_x_axis(axisLabel=list(interval=0)) %>%
          e_y_axis(name='Cell', nameLocation='center', 
                   axisLabel=list(show=FALSE), axisTick=list(show=FALSE),
                  ) %>%
          e_visual_map(v, text=c('umi', ''), orient='horizontal', top='top', left='center') %>%
          e_theme_custom(mytheme)

        Lres[[flt_mode]][['BD']] <- e_arrange(e1, e2, e3, e4, cols=2)
    }
}

In [ ]:
# VDJB

In [ ]:
for (flt_mode in names(Lres)) {
    if (('VDJB' %in% config[['flt_mode']][[flt_mode]]) && (config[[sample]][['VDJB']] == TRUE)) {
        
        # class
        TBflt <- Lres[[flt_mode]][['df']]
        tbclass <- TBflt %>% 
            fancy_count(class, ext=c_gene_H, ext_fmt='ratio') %>%
            kable

        TBp <- TBflt %>% count(clone=clone_cellranger) 
        Nclones <- TBp %>% filter(n > 1) %>% nrow
        Nclones_percent <- float_to_percent(Nclones / nrow(TBflt))

        e1 <- TBp %>%
          e_charts(clone) %>%
          e_pie(n, radius = c("50%", "70%"), 
                label=list(show=TRUE, position='center', 
                           formatter=Nclones_percent, fontSize=30), 
                itemStyle=list(borderColor='grey')) %>%
          e_legend(show=FALSE) %>% 
          pe_color(gradient_color(Vcol, n=seq_len(Nclones)), default=grey_col)

        e2 <- TBflt %>% select(class, v_domain_shm_ratio_H) %>%
          filter(!is.na(class)) %>%
          group_by(class) %>%
          e_chart() %>%
          e_density(v_domain_shm_ratio_H, areaStyle = list(opacity = 0.4), smooth = TRUE) %>%
          e_tooltip() %>%
          e_x_axis(name='Heavy Chain SHM', nameLocation='middle', nameGap=30) %>%
          e_y_axis(name='Density', nameLocation='middle', nameGap=30)

        e3 <- TBflt %>% select(class, v_domain_shm_ratio_L) %>%
          filter(!is.na(class)) %>%
          group_by(class) %>%
          e_chart() %>%
          e_density(v_domain_shm_ratio_L, areaStyle = list(opacity = 0.4), smooth = TRUE) %>%
          e_tooltip() %>%
          e_x_axis(name='Light Chain SHM', nameLocation='middle', nameGap=30) %>%
          e_y_axis(name='Density', nameLocation='middle', nameGap=30)

        Lres[[flt_mode]][['VDJB']] <- list(table=tbclass, plot=e_arrange(e1, e2, e3, cols=3))
    }
}

In [ ]:
# metadata

In [ ]:
TBmeta <- read_csv(snakemake@params[['metadata']])
Lres[['metadata']] <- TBmeta %>% 
    filter(sample==sample) %>% 
    remove_nacol %>% t %>% kable

In [ ]:
# cells

In [ ]:
flt_stat <- read_yaml(str_c(stat_dir, '/filter_stat.yaml'))

In [ ]:
Lres[['cells']] <- bind_cols(
    names(flt_stat) %>% as_tibble,
    flt_stat %>% map_dfr(~as_tibble(.x))
) %>% rename(item=value) %>%
    mutate(yield = float_to_percent(flt / raw, 2)) %>%
    kable

In [ ]:
# out

In [ ]:
out <- snakemake@output[['visualize_rds']]
dir.create(dirname(out), recursive = TRUE)
saveRDS(Lres, out)